In [1]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 10.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
import json
import os
import logging
import sys
import threading
import time

# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio


from llama_index.core import Document, QueryBundle
from llama_index.core.schema import TextNode
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.llms.groq import Groq

from langchain_groq import ChatGroq
from langchain.tools import tool, StructuredTool
from langchain_core.tools import ToolException
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

from groq import Groq, RateLimitError

from datasets import Dataset 

import spacy
import torch
import pandas as pd
import torch
import random
import ast
import ragas

from glob import glob
from dataclasses import dataclass
from typing import List, Dict
from thefuzz import process

from ragas.metrics import (
    faithfulness, 
    answer_relevancy, 
    context_relevancy, 
    answer_similarity, 
    answer_correctness,
)

from ragas import evaluate
from ragas.run_config import RunConfig

from datasets import load_dataset

from tqdm.notebook import tqdm

from rerankers import Reranker

In [3]:
print(ragas.__version__)

0.1.9


In [4]:
torch.set_default_device('cuda:0')
torch.cuda.get_device_name()

'NVIDIA GeForce GTX TITAN X'

In [5]:
random.seed(42)

In [6]:
nest_asyncio.apply()

os.environ["GROQ_API_KEY"] = "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"

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
SENTENCE_WINDOW_SIZE = 5
SENTENCE_WINDOW_STRIDE = 2

TOP_K = 4
BM25_TOP_K = 1000

RERANKER_MODEL = "unicamp-dl/monoptt5-base"

LLM_MODEL = "llama3-70b-8192"
LLM_CONTEXT_SIZE = 8192
LLM_TEMPERATURE = 0
LLM_TOP_P = 1

In [7]:
def get_transcription_documents(base_transcriptions_path):
    transcription_documents = []

    for transcriptions_path in glob(base_transcriptions_path):
        with open(transcriptions_path) as f:
            transcriptions = json.load(f)
        
        for transcription in transcriptions:
            transcription_documents.append(
                Document(
                    text=transcription['transcription'],
                    metadata={
                        'title': transcription['title'],
                        'publishing_date': transcription['publishing_date'],
                        'quadro': transcription['quadro'],
                        'hashtag': transcription['hashtag'],
                    },
                )
            )
            
    return transcription_documents

In [8]:
def sliding_window_split(documents, stride, window_size):
    sentencizer = spacy.blank('pt')
    sentencizer.add_pipe('sentencizer')
    
    window_documents = []

    for document in documents:
        text = document.text
        paragraphs = text.split('\n\n') 
        paragraphs = [ p.replace('\n', ' ').strip() for p in paragraphs if p.strip() ]
        for paragraph in paragraphs:
            p_sentencized = sentencizer(paragraph)
            sentences = [sent.text.strip() for sent in p_sentencized.sents]
            for i in range(0, len(sentences), stride):
                window_text = ' '.join(sentences[i : min(len(sentences), i+window_size)]).strip()
                window_metadata = document.metadata.copy()
                window_metadata['parent_document_id'] = document.id_
                window_documents.append(Document(text=window_text, metadata=window_metadata))

    return window_documents

In [9]:
def get_transcriptions_nodes(transcription_documents):
    transcription_window_documents = sliding_window_split(transcription_documents, SENTENCE_WINDOW_STRIDE, SENTENCE_WINDOW_SIZE)
    
    transcription_nodes = dict()
    for document in transcription_window_documents:
        new_node = TextNode(id=document.id_, text=document.text, metadata=document.metadata)
        if new_node.metadata['title'] not in transcription_nodes.keys():
            transcription_nodes[document.metadata['title']] = [new_node]
        else:
            transcription_nodes[document.metadata['title']].append(new_node)
    
    return transcription_nodes

In [10]:
transcription_documents = get_transcription_documents("../transcriptions-headless/*.json")

In [11]:
transcription_nodes = get_transcriptions_nodes(transcription_documents)

In [12]:
class GroqClient:
    '''
    Interface for using the Groq API

    Implements a rate limit control for multi-threading use.
    '''

    # documentacao dos parametros em: https://console.groq.com/docs/text-chat
    _context_size = LLM_CONTEXT_SIZE
    _temperature = LLM_TEMPERATURE
    _top_p = LLM_TOP_P
    _stop = None
    _stream = False

    # Mutex lock
    _rate_lock = threading.Lock()

    def __init__(self, llm_model, api_key=os.environ['GROQ_API_KEY']):
        '''
        GroqClient constructor.
        '''
        self._client = Groq(api_key=api_key)
        self._llm_model = llm_model


    def __call__(self, prompt: str) -> str:
        '''
        Generates the model response

        Args:
            prompt (str): prompt to send to the model.

        Returns:
            str: model response.
        '''

        done = False
        while not done:
            try:
                GroqClient._rate_lock.acquire()
                GroqClient._rate_lock.release()
                chat_completion = self._client.chat.completions.create(
                    messages=[
                        {
                            "role": "system",
                            "content": prompt,
                        }
                    ],
                    model=self._llm_model,
                    temperature=self._temperature,
                    max_tokens=self._context_size,
                    top_p=self._top_p,
                    stop=self._stop,
                    stream=self._stream,
                )
                done = True
            except RateLimitError as exception:
                GroqClient.error = exception
                if not GroqClient._rate_lock.locked():
                    GroqClient._rate_lock.acquire()
                    time.sleep(1.75)
                    GroqClient._rate_lock.release()

        return chat_completion.choices[0].message.content

In [13]:
@dataclass
class RAGResponse:
    answer: str
    contexts: List[str]

In [14]:
class MultiIndexRetriever:
    def __init__(self, indexes_nodes: Dict[str, List[TextNode]], top_k: int, bm25_top_k: int, reranker_model: str) -> None:
        self.indexes: Dict[str, List[TextNode]] = indexes_nodes
        
        self.bm25_retrievers: Dict[str, BM25Retriever] = dict()
        for index, nodes in indexes_nodes.items():
            self.bm25_retrievers[index] = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=bm25_top_k)

        self.reranker = ranker = Reranker(
            reranker_model,
            inputs_template="Pergunta: {query} Documento: {text} Relevante:",
            dtype=torch.float32
        )
        self.top_k = top_k
        

    def retrieve(self, index_name: str, query: str):
        if index_name not in self.indexes.keys():
            raise ValueError(f"Index {index_name} not found")
        
        retriever = self.bm25_retrievers[index_name]

        retrieved_nodes = retriever.retrieve(query)
        retrieved_chunks = [ node.get_text() for node in retrieved_nodes ]

        ranking = self.reranker.rank(query, retrieved_chunks)
        context_chunks = [ doc.text for doc in ranking.results[:self.top_k] ]

        return context_chunks

In [15]:
class IndexDetector:
    def __init__(self, llm_model: str, index_names: List[str], api_key: str = os.environ['GROQ_API_KEY']):
        self._base_prompt = \
            "Given the following Portuguese query, regarding an episode of a podcast, please tell me the title of the episode. " \
            "The query starts now: '{query}'." \
            "You MUST answer with only the title of the episode."
        
        self.llm = GroqClient(llm_model=llm_model, api_key=api_key)
        self.llm_model_name = llm_model
        self.index_names = index_names

    def detect_index(self, query: str) -> str:
        prompt = self._base_prompt.format(query=query)
        raw_llm_guess = self.llm(prompt)
        llm_guess = raw_llm_guess.strip().lower()

        if llm_guess in self.index_names:
            index_name = llm_guess
        else:
            index_name, _ = process.extractOne(llm_guess, self.index_names)
            
        return index_name

In [16]:
# Prompt adaptado a partir da combinação dos seguintes prompts:
#   - https://github.com/run-llama/llama_index/blob/a87b63fce3cc3d24dc71ae170a8d431440025565/llama_index/agent/react/prompts.py
#   - https://smith.langchain.com/hub/hwchase17/react-chat

REACT_CHAT_SYSTEM_HEADER = """\
You are designed to help answering questions regarding the content of the Não Inviabilize podcast episodes.
You are a Large Language Model trained by Meta AI. As a language model, you are able to generate human-like text based on the input you receive, allowing yourself to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions.
Additionally, you are able to generate your own text based on the input it receives, allowing yourself to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, you are a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
When the user needs help with a specific question about something that was discussed on an specific episode of the Não Inviabilize podcast, you are here to assist.

TOOLS:
------
You have access to a variety of tools that can help you get information to answer questions.
You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.
You have access to the following tools:
{tools}

To use a tool, please use the following format:
```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```
But be careful, you MUST answer the question in Portuguese only!

If there is not enough information in the context passages, answer "Não há informação suficiente no episódio."

Begin!
User question: {input}

{agent_scratchpad}
"""

class ReActRAGPipeline:
    def __init__(
        self,
        retriever: MultiIndexRetriever, 
        index_detector: IndexDetector,
        verbose: bool = False,
    ) -> None:
        self.index_detector: IndexDetector = index_detector
        self.retriever: MultiIndexRetriever = retriever

        base_prompt = PromptTemplate.from_template(REACT_CHAT_SYSTEM_HEADER)
        llm = ChatGroq(
            temperature=LLM_TEMPERATURE,
            model_name=LLM_MODEL,
            api_key=os.environ['GROQ_API_KEY']
        )

        tools = [
            StructuredTool.from_function(
                func=self._search_relevant_passages,
                name="Search Relevant Contexts Passages",
                description="Returns a sequence of relevant contexts passages for the given question about a specific episode in the podcast. In order to perform this search, you must provide the question in Portuguese.",
                handle_tool_error=ReActRAGPipeline._handle_error,
            ),
        ]
        agent = create_react_agent(llm, tools, base_prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose, handle_parsing_errors=True)

        self._search_history = []

        self.agent_executor = agent_executor

    def _handle_error(error: ToolException) -> str:
        return (
            "The following errors occurred during tool execution:"
            + error.args[0]
            + "Please try another tool."
        )
    
    def _search_relevant_passages(self, question: str) -> list[str]:
        """Returns a sequence of relevant contexts passages for the given question about a specific episode in the podcast. In order to perform this search, you must provide the question in Portuguese."""
        episode_title = self.index_detector.detect_index(question)
        contexts = self.retriever.retrieve(episode_title, question)
        self._search_history += contexts
        return contexts
    
    def __call__(self, question: str) -> RAGResponse:
        self._search_history = []

        output = self.agent_executor.invoke({"input": question})
        answer = output['output']
        contexts = self._search_history[:]
        
        self._search_history = []

        return RAGResponse(answer, contexts)

In [17]:
index_detector = IndexDetector(llm_model=LLM_MODEL, index_names=list(transcription_nodes.keys()), api_key=os.environ['GROQ_API_KEY'])
index_name = index_detector.detect_index('No episódio \'restropectiva 2022 + bônus da história portugal\', quem é o marido da Lúcia na história \"Parmegiana\"?')
index_name

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'restropectiva 2022 + bônus da história portugal'

In [18]:
multi_index_retriever = MultiIndexRetriever(
    indexes_nodes=transcription_nodes,
    top_k=TOP_K,
    bm25_top_k=BM25_TOP_K,
    reranker_model=RERANKER_MODEL
)
ret = multi_index_retriever.retrieve('restropectiva 2022 + bônus da história portugal', 'No episódio \'restropectiva 2022 + bônus da história portugal\', quem é o marido da Lúcia na história \"Parmegiana\"?')
ret

Loading T5Ranker model unicamp-dl/monoptt5-base
No device set
Using device cuda
Using dtype torch.float32
Loading model unicamp-dl/monoptt5-base, this might take a while...
Using device cuda.
Using dtype torch.float32.


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5 true token set to ▁Sim
T5 false token set to ▁Não
Returning normalised scores...
Inputs template set to Pergunta: {query} Documento: {text} Relevante:


Scoring...:   0%|          | 0/5 [00:00<?, ?it/s]

['Bom, em décimo lugar… Agora vamos às histórias. Em décimo lugar, os nossos ouvintes escolheram aí a história “Parmegiana”. Então, se você não ouviu, essa é uma história que está em todas as plataformas gratuitas, é só você jogar na busca aí ou no Spotify, Deezer, Amazon, enfim, onde você quiser, a gente está… E que era a história Parmegiana? A história Parmegiana é a história da Lúcia, que era casada aí e o marido dela sempre fazia uma parmegiana especial para a Lúcia. Era um mimo… Então, até quando eles brigavam ou, sei lá, quando eu tinha um momento especial, esse marido da Lúcia fazia essa parmegiana pra ela.',
 'Então, se você não ouviu, essa é uma história que está em todas as plataformas gratuitas, é só você jogar na busca aí ou no Spotify, Deezer, Amazon, enfim, onde você quiser, a gente está… E que era a história Parmegiana? A história Parmegiana é a história da Lúcia, que era casada aí e o marido dela sempre fazia uma parmegiana especial para a Lúcia. Era um mimo… Então, até

In [19]:
react_rag_pipeline = ReActRAGPipeline(
    retriever=multi_index_retriever, 
    index_detector=index_detector,
)
react_rag_pipeline("No episódio 'bia', quem é a Bia e o que aconteceu com ela?")

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Scoring...:   0%|          | 0/3 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


RAGResponse(answer='A Bia é uma voluntária de uma ONG de apoio a pacientes com câncer e palestrante que cria conteúdos relacionados ao tema nas redes sociais. Ela foi diagnosticada com câncer colorretal e passou por tratamento. Além disso, ela é ostomizada e usa uma bolsinha para coletar fezes.', contexts=['Atualmente, a Bia segue fazendo acompanhamento a cada seis meses, com exames de imagem e de sangue e a Bia é voluntária de uma ONG de apoio. Essa ONG presta informação e defesa dos direitos dos pacientes com câncer. Ela já está nessa ONG como voluntária há quatro anos e ela é palestrante e cria conteúdos também relacionados ao tema aí nas redes sociais. Eu vou deixar o perfil da Bia aqui na descrição do episódio. E, pra Bia, falar de prevenção e diagnóstico precoce é um ato de amor ao próximo, né?', 'Ela já está nessa ONG como voluntária há quatro anos e ela é palestrante e cria conteúdos também relacionados ao tema aí nas redes sociais. Eu vou deixar o perfil da Bia aqui na descriç

In [8]:
langchain_embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-cos-v1")

langchain_groq_completion = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=os.environ['GROQ_API_KEY']
)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Use pytorch device_name: cuda
Load pretrained SentenceTransformer: sentence-transformers/multi-qa-mpnet-base-cos-v1


In [9]:
from __future__ import print_function

import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread
    
try: # use code that works the same in Python 2 and 3
    range, _print = xrange, print
    def print(*args, **kwargs): 
        flush = kwargs.pop('flush', False)
        _print(*args, **kwargs)
        if flush:
            kwargs.get('file', sys.stdout).flush()            
except NameError:
    pass



def cdquit(fn_name):
    # print to stderr, unbuffered in Python 2.
    print('{0} took too long'.format(fn_name), file=sys.stderr)
    sys.stderr.flush() # Python 3 stderr is likely buffered.
    thread.interrupt_main() # raises KeyboardInterrupt
    
def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, cdquit, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

In [10]:
@exit_after(600)
def evaluate_ragas(llm_model, embeddings, answers_dataset):
    run_config = RunConfig(
        timeout = 60,
        max_retries = 1000,
        max_wait = 120,
        max_workers = 2,
    )
    score = evaluate(
        answers_dataset,
        metrics=[answer_relevancy, faithfulness, context_relevancy, answer_similarity, answer_correctness],
        llm=llm_model,
        embeddings=embeddings,
        run_config=run_config,
    )
    score = score.to_pandas()
    return score

In [23]:
episodes_file = open("episodes.json", "r")
episodes = json.load(episodes_file)
episodes_file.close()

print(f'{len(episodes)} episodios')

75 episodios


In [24]:
perguntas = []
respostas = []
contextos = []
gts = []
quadros = []
for ep in tqdm(episodes):
    try:
        if "Perguntas" in ep.keys() and "Respostas" in ep.keys():
            for pergunta, gt in zip(ep["Perguntas"], ep["Respostas"]):
                try:
                    rag_answer = react_rag_pipeline(pergunta)
                    perguntas.append(pergunta)
                    respostas.append(rag_answer.answer)
                    contextos.append(rag_answer.contexts)
                    gts.append(gt)
                    quadros.append(quadro)
                    print(f"{len(perguntas)} perguntas respondidas")
                except Exception as e2:
                    print(f"Erro {e2} no loop interno")
    except Exception as e3:
            print(f"Erro {e3} no loop externo")

  0%|          | 0/75 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Scoring...:   0%|          | 0/3 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Erro name 'quadro' is not defined no loop interno
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Scoring...:   0%|          | 0/3 [00:00<?, ?it/s]

...

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Erro name 'quadro' is not defined no loop interno
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 5.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Scoring...:   0%|          | 0/2 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 12.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Erro name 'quadro' is not defined no loop interno
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Scoring...:   0%|          | 0/2 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 15.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Erro name 'quadro' is not defined no loop interno


In [33]:
answers_data = {
    'question': perguntas,
    'answer': gts if respostas is None else respostas,
    'ground_truth': gts,
    'contexts': contextos,
}

In [35]:
df_answers = pd.DataFrame(answers_data)
df_answers.to_csv('react_answers.csv')

In [11]:
df_answers = pd.read_csv('react_answers.csv')
df_answers = df_answers.drop('Unnamed: 0', axis=1)
df_answers['contexts'] = df_answers['contexts'].apply(lambda x: ast.literal_eval(x))
df_answers.head()

,question,answer,ground_truth,contexts
0,"No episódio 'imundo', o que a Luciana descobri...",A Luciana descobriu que seu marido era muito d...,Um lado preguiçoso e desleal do marido.,[Era um lado do marido que a Luciana não conhe...
1,"No episódio 'imundo', por que a Luciana decidi...",A Luciana decidiu pedir o divórcio porque o ma...,Porque ele não ajudava em nada em casa e a tra...,[Então isso revolta realmente. E aí com dois m...
2,"No episódio 'imundo', como os amigos comuns re...",Os amigos comuns ficaram chocados e não acredi...,Romperam com ela ou ficaram calados.,[Então isso revolta realmente. E aí com dois m...
3,"No episódio 'analu', quem é Ana Luísa?","Ana Luísa é uma moça muito simpática, mãe solo...",Ana Luísa é uma mãe solo que mudou de carreira...,[A história da Ana Luísa é uma entre as histór...
4,"No episódio 'analu', o que é o projeto Cloud G...",O projeto Cloud Girls é um projeto que proporc...,O projeto Cloud Girls é uma iniciativa que pro...,[Em três episódios eu vou contar histórias sob...


In [16]:
# df_ragas = None

for idx, row in df_answers.iterrows():
    sample = dict()
    for key, val in row.to_dict().items():
        sample[key] = [val]
    
    dt = Dataset.from_dict(sample)

    retries = 3
    done = False
    while not done and retries > 0:
        try:
            retries -= 1
            batch_df_ragas = evaluate_ragas(langchain_groq_completion, langchain_embeddings_model, dt)
            done = True
        except Exception as e:
            if retries > 0:
                print(f'Erro {e}')
                time.sleep(30)
        
    if not df_ragas is None:
        df_ragas = pd.concat([df_ragas, batch_df_ragas], ignore_index=True)
    else:
        df_ragas = batch_df_ragas

    print(f'{len(df_ragas)} perguntas avaliadas')

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 29.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Retrying request to /openai/v1/chat/completions in 0.840143 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 9.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 8.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 22.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 24.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Man

/opt/conda/lib/python3.10/site-packages/ragas/evaluation.py:299: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


...

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Retrying request to /openai/v1/chat/completions in 0.867270 seconds
Retrying request to /openai/v1/chat/completions in 0.893624 seconds
Retrying request to /openai/v1/chat/completions in 1.627032 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 30.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 28.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST h

evaluate_ragas took too long


HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 23.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 23.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 23.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 23.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /ope

KeyboardInterrupt: 

In [17]:
df_ragas.to_csv('partial_react_ragas_eval_178.csv')

In [12]:
df_ragas = pd.read_csv('partial_react_ragas_eval_178.csv')
df_ragas = df_ragas.drop('Unnamed: 0', axis=1)
df_ragas

,question,answer,ground_truth,contexts,answer_relevancy,faithfulness,context_relevancy,answer_similarity,answer_correctness
0,"No episódio 'imundo', o que a Luciana descobri...",A Luciana descobriu que seu marido era muito d...,Um lado preguiçoso e desleal do marido.,['Era um lado do marido que a Luciana não conh...,0.369835,1.00,0.272727,0.627742,0.156936
1,"No episódio 'imundo', por que a Luciana decidi...",A Luciana decidiu pedir o divórcio porque o ma...,Porque ele não ajudava em nada em casa e a tra...,['Então isso revolta realmente. E aí com dois ...,0.613711,1.00,0.222222,0.521067,NaN
2,"No episódio 'imundo', como os amigos comuns re...",Os amigos comuns ficaram chocados e não acredi...,Romperam com ela ou ficaram calados.,['Então isso revolta realmente. E aí com dois ...,0.401724,0.75,0.277778,0.487562,NaN
3,"No episódio 'analu', quem é Ana Luísa?","Ana Luísa é uma moça muito simpática, mãe solo...",Ana Luísa é uma mãe solo que mudou de carreira...,['A história da Ana Luísa é uma entre as histó...,0.614053,1.00,0.277778,0.852256,0.588064
4,"No episódio 'analu', o que é o projeto Cloud G...",O projeto Cloud Girls é um projeto que proporc...,O projeto Cloud Girls é uma iniciativa que pro...,['Em três episódios eu vou contar histórias so...,0.566112,1.00,0.200000,0.972070,NaN
...,...,...,...,...,...,...,...,...,...
173,"No episódio 'casamentão', quem se tornou o mar...","No final da história, Karina se casou com o Do...",Doutor Murilo,"['Déia Freitas: Então é isso, gente, essa é a ...",0.440268,1.00,0.230769,0.541658,0.885415
174,"No episódio 'balão', quem é o personagem princ...",O personagem principal da história é a Cíntia.,Cíntia,['Detalhe: Ele tinha pedido pro cara do balão ...,0.334582,1.00,0.333333,0.416723,0.854181
175,"No episódio 'balão', o que era a surpresa do n...",A surpresa do namorado de Cíntia era um passei...,Um passeio de balão em Boituva,['E aí ela estava toda enjoada ainda… Ela viro...,0.269677,1.00,0.250000,0.524893,0.631223
176,"No episódio 'balão', como terminou o relaciona...",O relacionamento de Cíntia e seu namorado term...,"Com um término, após uma briga sobre o pagamen...",['E aí falou que estava repensando e aí a Cínt...,0.244995,1.00,0.250000,0.453427,0.446690


In [13]:
df_answers

,question,answer,ground_truth,contexts
0,"No episódio 'imundo', o que a Luciana descobri...",A Luciana descobriu que seu marido era muito d...,Um lado preguiçoso e desleal do marido.,[Era um lado do marido que a Luciana não conhe...
1,"No episódio 'imundo', por que a Luciana decidi...",A Luciana decidiu pedir o divórcio porque o ma...,Porque ele não ajudava em nada em casa e a tra...,[Então isso revolta realmente. E aí com dois m...
2,"No episódio 'imundo', como os amigos comuns re...",Os amigos comuns ficaram chocados e não acredi...,Romperam com ela ou ficaram calados.,[Então isso revolta realmente. E aí com dois m...
3,"No episódio 'analu', quem é Ana Luísa?","Ana Luísa é uma moça muito simpática, mãe solo...",Ana Luísa é uma mãe solo que mudou de carreira...,[A história da Ana Luísa é uma entre as histór...
4,"No episódio 'analu', o que é o projeto Cloud G...",O projeto Cloud Girls é um projeto que proporc...,O projeto Cloud Girls é uma iniciativa que pro...,[Em três episódios eu vou contar histórias sob...
...,...,...,...,...
220,"No episódio 'fritadeira', por que a Estela não...",A Estela não gostava da Marcela porque a Marce...,A Estela não gostava da Marcela porque ela que...,"[E, um dia, a Estela, essa menina chata e mima..."
221,"No episódio 'fritadeira', por que o Júnior ped...",O Júnior pediu o divórcio da Marcela porque el...,O Júnior pediu o divórcio da Marcela porque el...,"[E aí, gente, mais uns dias aí depois que ela ..."
222,"No episódio 'corrida', quem é o personagem pri...",O personagem principal da história é o Roni.,O Roni,"[[trilha], Não sei, se acontecer comigo eu vou..."
223,"No episódio 'corrida', por que o Roni começou ...",O Roni começou a correr de madrugada porque nã...,Porque ele começou a sair pra correr sozinho e...,"[O Roni mora sozinho, tá sem nenhuma namorada,..."


In [14]:
for idx, row in df_answers.iloc[len(df_ragas):].iterrows():
    sample = dict()
    for key, val in row.to_dict().items():
        sample[key] = [val]
    
    dt = Dataset.from_dict(sample)

    retries = 3
    done = False
    while not done and retries > 0:
        try:
            retries -= 1
            batch_df_ragas = evaluate_ragas(langchain_groq_completion, langchain_embeddings_model, dt)
            done = True
        except Exception as e:
            if retries > 0:
                print(f'Erro {e}')
                time.sleep(30)
        
    df_ragas = pd.concat([df_ragas, batch_df_ragas], ignore_index=True)

    print(f'{len(df_ragas)} perguntas avaliadas')

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 11.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 11.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 11.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Retrying request to /openai/v1/chat/completions in 0.840143 seconds
Retrying request to /openai/v1/chat/completions in 0.993747 seconds
Retrying request to /openai/v1/chat/completions in 1.862485 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 33.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 30.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 5.000000 seconds
HTTP 

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

...

Retrying request to /openai/v1/chat/completions in 0.865176 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 9.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 9.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 9.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 6.000000 seconds
HTTP Re

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Retrying request to /openai/v1/chat/completions in 0.751213 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 11.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 11.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 5.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 10.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
HTTP

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Retrying request to /openai/v1/chat/completions in 0.870606 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 10.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 5.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 17.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Man

In [15]:
df_ragas.head()

,question,answer,ground_truth,contexts,answer_relevancy,faithfulness,context_relevancy,answer_similarity,answer_correctness
0,"No episódio 'imundo', o que a Luciana descobri...",A Luciana descobriu que seu marido era muito d...,Um lado preguiçoso e desleal do marido.,['Era um lado do marido que a Luciana não conh...,0.369835,1.00,0.272727,0.627742,0.156936
1,"No episódio 'imundo', por que a Luciana decidi...",A Luciana decidiu pedir o divórcio porque o ma...,Porque ele não ajudava em nada em casa e a tra...,['Então isso revolta realmente. E aí com dois ...,0.613711,1.00,0.222222,0.521067,NaN
2,"No episódio 'imundo', como os amigos comuns re...",Os amigos comuns ficaram chocados e não acredi...,Romperam com ela ou ficaram calados.,['Então isso revolta realmente. E aí com dois ...,0.401724,0.75,0.277778,0.487562,NaN
3,"No episódio 'analu', quem é Ana Luísa?","Ana Luísa é uma moça muito simpática, mãe solo...",Ana Luísa é uma mãe solo que mudou de carreira...,['A história da Ana Luísa é uma entre as histó...,0.614053,1.00,0.277778,0.852256,0.588064
4,"No episódio 'analu', o que é o projeto Cloud G...",O projeto Cloud Girls é um projeto que proporc...,O projeto Cloud Girls é uma iniciativa que pro...,['Em três episódios eu vou contar histórias so...,0.566112,1.00,0.200000,0.972070,NaN


In [18]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [19]:
df_ragas.describe().drop(['count'], axis=0)

,answer_relevancy,faithfulness,context_relevancy,answer_similarity,answer_correctness
mean,0.336,0.829,0.262,0.571,0.468
std,0.157,0.306,0.078,0.196,0.297
min,0.000,0.000,0.048,0.042,0.010
25%,0.245,0.667,0.222,0.454,0.149
50%,0.336,1.000,0.263,0.565,0.479
75%,0.440,1.000,0.308,0.694,0.697
max,0.745,1.000,0.500,1.000,1.000


In [17]:
df_ragas.to_csv('react_ragas_eval.csv')